In [1]:
# !wget https://huggingface.co/datasets/mesolitica/msa-common-crawl-qa/resolve/main/common-crawl-qa.json

In [2]:
import json

with open('common-crawl-qa.json') as fopen:
    common_crawl = json.load(fopen)

In [3]:
import malaya

model = malaya.augmentation.abstractive.huggingface(model = 'mesolitica/translation-t5-small-standard-bahasa-cased')

2023-07-25 22:30:27.276858: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 22:30:27.363335: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-25 22:30:28.095029: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-25 22:30:28.095074: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [4]:
_ = model.cuda()
_ = model.model.eval()

In [5]:
def generate_noisy(text):
    g = model.generate([text],to_lang = 'pasar ms',
              max_length = 512, do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
              num_return_sequences=5)
    result = []
    for g_ in g:
        splitted = g_.split()
        splitted = [s for s in splitted if s[0] !='@']
        s = ' '.join(splitted)
        if len(s):
            result.append(s)
    return result

generate_noisy("Apakah makna nama 'Guinness'?")

["Apa maknanya nama 'Guinness'?",
 'Apa makna nama "Guinness"?',
 "Apa maksud nama 'Guinness' ni?",
 'Gandness ni apa maknanya?']

In [6]:
template_malaya = {
    'description': 'Template used by Malaya.',
    'prompt_input': 'Di bawah ialah arahan yang menerangkan tugasan, termasuk dengan input yang menyediakan konteks lanjut. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Input:\n{input}\n\n### Jawapan:\n',
    'prompt_no_input': 'Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n',
    'response_split': '### Jawapan:',
}

In [12]:
from tqdm import tqdm
import random

data_common_crawl = []
for d in tqdm(common_crawl):
    try:
        t = d[0]
        a = d[1]
        j_a = json.loads(d[1])
        q = t
        
        for r in j_a['soalan']:
            if random.random() > 0.5:
                g = generate_noisy(r['soalan'])
                soalan = random.choice(g)
            else:
                soalan = r['soalan']
                
            if random.random() > 0.5:
                soalan = soalan.lower()
            
            if random.random() > 0.7:
                q_ = f"paragraph `{q}`, berdasarkan paragraph, jawab soalan `{soalan}`"
            else:
                q_ = f"paragraph `{q}`, {soalan}"
                
            data = {
                'prompt_input': template_malaya["prompt_no_input"],
                'input': q_,
                'output': r['jawapan']
            }
            data_common_crawl.append(data)
        
        k = '({"soalan": [{`soalan`, `jawapan`}]})'
        p = f"""
paragraph, `{t}`
hasilkan 3 soalan dan jawapan khusus berdasarkan perenggan itu, kembalikan sebagai struktur JSON {k}
        """
        data = {
            'prompt_input': template_malaya["prompt_no_input"],
            'input': p,
            'output': a
        }
        data_common_crawl.append(data)

    except Exception as e:
        print(e)
        pass

  0%|                                                                                                          | 0/77002 [00:00<?, ?it/s]

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not 

  0%|▎                                                                                             | 295/77002 [02:14<9:11:01,  2.32it/s]

Extra data: line 18 column 1 (char 829)
Extra data: line 18 column 1 (char 790)
Extra data: line 18 column 1 (char 617)
Extra data: line 18 column 1 (char 665)
Extra data: line 18 column 1 (char 752)


  0%|▍                                                                                            | 384/77002 [03:05<15:21:37,  1.39it/s]

...

100%|██████████████████████████████████████████████████████████████████████████████████████████▉| 76996/77002 [10:49:10<00:01,  3.83it/s]

Extra data: line 18 column 1 (char 659)


100%|██████████████████████████████████████████████████████████████████████████████████████████▉| 77000/77002 [10:49:11<00:00,  4.64it/s]

Extra data: line 18 column 1 (char 853)
Extra data: line 18 column 1 (char 947)


100%|███████████████████████████████████████████████████████████████████████████████████████████| 77002/77002 [10:49:11<00:00,  1.98it/s]


In [14]:
with open('prepared-common-crawl-qa.jsonl', 'w') as fopen:
    for l in data_common_crawl:
        fopen.write(f'{json.dumps(l)}\n')